In [1]:
from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI
from langchain.tools import DuckDuckGoSearchRun
from crewai_tools import DirectoryReadTool,FileReadTool
from langchain.tools.file_management.read import ReadFileTool
from langchain_anthropic import ChatAnthropic
from dotenv import load_dotenv
import os

load_dotenv()

gpt = ChatOpenAI(model="gpt-4o-mini",
             api_key=os.getenv("OPENAI_API_KEY"), 
             temperature=0,
             max_tokens=4000)

clud = ChatAnthropic(model="claude-3-5-sonnet-20240620",
                     anthropic_api_key=os.getenv("Anthropic_API_KEY"),
                     temperature=0,
                     max_tokens=4000)

# 도구 초기화
search_tool = DuckDuckGoSearchRun()

# 에이전트 정의
travel_planner = Agent(
    role='여행 계획가',
    goal='고객의 선호도와 예산에 맞는 최적의 여행 계획 수립',
    backstory='15년 경력의 전문 여행 계획가로, 다양한 고객의 요구사항을 만족시키는 맞춤형 여행 계획을 수립합니다.',
    verbose=True,
    allow_delegation=True,
    tools=[search_tool],
    llm=gpt
)

local_expert = Agent(
    role='도쿄 현지 전문가',
    goal='도쿄의 숨겨진 명소와 현지 문화에 대한 깊이 있는 정보 제공',
    backstory='도쿄에서 10년 이상 거주한 현지 전문가로, 관광객들이 쉽게 접하기 어려운 특별한 경험을 제안합니다.',
    verbose=True,
    allow_delegation=True,
    tools=[search_tool],
    llm=gpt
)

schedule_optimizer = Agent(
    role='일정 최적화 전문가',
    goal='효율적이고 즐거운 여행 일정 최적화',
    backstory='데이터 분석과 여행 경험을 결합하여 고객의 시간과 에너지를 최대한 활용할 수 있는 최적의 일정을 설계합니다.',
    verbose=True,
    allow_delegation=True,
    tools=[search_tool],
    llm=gpt
)

# 여행 정보 딕셔너리
travel_info = {
    "destination": "일본 도쿄",
    "duration": "3일",
    "travelers": "와이프와 둘이",
    "arrival": "저녁 8시",
    "hotels": {
        "day1": "도세이호텔 코코네우에노",
        "day2_3": "Nippon Seinenkan Hotel Tokyo, Shinjuku"
    },
    "departure": "3일차 저녁 10시",
    "preferences": "유명관광지, 면세 쇼핑, 적절히 맛있는 음식"
}

# 태스크 정의
task1 = Task(
    description=f'''
    다음 여행 정보를 분석하고 고객의 선호도와 제약 사항을 파악하세요:
    - 목적지: {travel_info['destination']}
    - 기간: {travel_info['duration']}
    - 여행자: {travel_info['travelers']}
    - 도착 시간: {travel_info['arrival']}
    - 호텔: 1일차 {travel_info['hotels']['day1']}, 2-3일차 {travel_info['hotels']['day2_3']}
    - 출발 시간: {travel_info['departure']}
    - 선호사항: {travel_info['preferences']}
    ''',
    expected_output="고객의 여행 선호도와 제약 사항에 대한 상세한 분석",
    agent=travel_planner
)

task2 = Task(
    description=f'''
    {travel_info['destination']}의 추천 장소와 활동 목록을 작성하세요. 
    특히 {travel_info['preferences']}에 중점을 두고, {travel_info['duration']} 동안 
    {travel_info['travelers']}이 즐길 수 있는 장소를 추천해주세요.
    ''',
    expected_output="도쿄의 추천 장소와 활동 목록 (휴양 및 맛집 중심)",
    agent=local_expert
)

task3 = Task(
    description=f'''
    주어진 정보와 추천 목록을 바탕으로 {travel_info['duration']}간의 세부 여행 일정을 최적화하세요.
    - 도착: {travel_info['arrival']}
    - 1일차 호텔: {travel_info['hotels']['day1']}
    - 2-3일차 호텔: {travel_info['hotels']['day2_3']}
    - 출발: {travel_info['departure']}
    선호사항인 {travel_info['preferences']}을 고려하여 일정을 조정하세요.
    각 동선 별 이동경로, 이동수단을 상세히 알려주세요.
    음식점은 끼니별로 예비후보지 2~3곳도 추가로 알려주세요
    ''',
    expected_output="3일간의 최적화된 세부 여행 일정",
    agent=schedule_optimizer
)

# Crew 생성 및 실행
crew = Crew(
    agents=[travel_planner, local_expert, schedule_optimizer],
    tasks=[task1, task2, task3],
    verbose=1
)

result = crew.kickoff()

print("최종 여행 계획:")
print(result)

 [DEBUG]: == Working Agent: 여행 계획가


> Entering new CrewAgentExecutor chain...
Thought: 고객의 여행 정보를 분석하여 선호도와 제약 사항을 파악해야 합니다. 고객은 일본 도쿄를 3일 동안 여행하며, 저녁 8시에 도착하고, 3일차 저녁 10시에 출발합니다. 호텔은 1일차 도세이호텔 코코네우에노, 2-3일차 Nippon Seinenkan Hotel Tokyo, Shinjuku입니다. 고객의 선호사항은 유명 관광지, 면세 쇼핑, 적절히 맛있는 음식입니다. 이 정보를 바탕으로 고객의 요구를 충족할 수 있는 여행 계획을 수립할 수 있도록 분석하겠습니다.

Action: Delegate work to co-worker
Action Input: {
    "coworker": "도쿄 현지 전문가",
    "task": "고객의 여행 선호도와 제약 사항에 맞는 관광지 및 음식점 추천",
    "context": "고객은 일본 도쿄를 3일 동안 여행하며, 저녁 8시에 도착하고, 3일차 저녁 10시에 출발합니다. 호텔은 1일차 도세이호텔 코코네우에노, 2-3일차 Nippon Seinenkan Hotel Tokyo, Shinjuku입니다. 고객의 선호사항은 유명 관광지, 면세 쇼핑, 적절히 맛있는 음식입니다."
} 

I encountered an error while trying to use the tool. This was the error: StructuredTool._run() missing 1 required keyword-only argument: 'config'.
 Tool Delegate work to co-worker accepts these inputs: Delegate a specific task to one of the following co-workers: [도쿄 현지 전문가, 일정 최적화 전문가]
The input to this tool should be the coworker, th

KeyboardInterrupt: 

In [1]:
from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI
from langchain.tools import DuckDuckGoSearchRun
from langchain_anthropic import ChatAnthropic
from dotenv import load_dotenv
import os

load_dotenv()

gpt = ChatOpenAI(model="gpt-4o-mini",
             api_key=os.getenv("OPENAI_API_KEY"), 
             temperature=0,
             max_tokens=4000)

clud = ChatAnthropic(model="claude-3-5-sonnet-20240620",
                     anthropic_api_key=os.getenv("Anthropic_API_KEY"),
                     temperature=0,
                     max_tokens=4000)

# 도구 초기화
search_tool = DuckDuckGoSearchRun()

# 에이전트 정의
travel_planner = Agent(
    role='여행 계획가',
    goal='고객의 선호도와 예산에 맞는 최적의 여행 계획 수립',
    backstory='15년 경력의 전문 여행 계획가로, 다양한 고객의 요구사항을 만족시키는 맞춤형 여행 계획을 수립합니다.',
    verbose=True,
    allow_delegation=False,
    tools=[search_tool],
    llm=gpt
)

local_expert = Agent(
    role='도쿄 현지 전문가',
    goal='도쿄의 숨겨진 명소와 현지 문화에 대한 깊이 있는 정보 제공',
    backstory='도쿄에서 10년 이상 거주한 현지 전문가로, 관광객들이 쉽게 접하기 어려운 특별한 경험을 제안합니다.',
    verbose=True,
    allow_delegation=False,
    tools=[search_tool],
    llm=clud
)

schedule_optimizer = Agent(
    role='일정 최적화 전문가',
    goal='효율적이고 즐거운 3일 도쿄 여행 일정 최적화',
    backstory='데이터 분석과 여행 경험을 결합하여 고객의 시간과 에너지를 최대한 활용할 수 있는 최적의 일정을 설계합니다.',
    verbose=True,
    allow_delegation=False,
    tools=[search_tool],
    llm=clud
)

# 여행 정보 딕셔너리
travel_info = {
    "destination": "일본 도쿄",
    "duration": "3일",
    "travelers": "와이프와 둘이",
    "arrival": "저녁 8시",
    "hotels": {
        "day1": "도세이호텔 코코네우에노",
        "day2_3": "Nippon Seinenkan Hotel Tokyo, Shinjuku"
    },
    "departure": "3일차 저녁 10시",
    "preferences": "유명관광지, 면세 쇼핑, 적절히 맛있는 음식"
}

# 고객 선호도 파악 및 우선순위 설정 함수
def get_customer_preferences():
    preferences = {}
    weights = {}
    
    food_preferences = [
        "1. 일본 전통 요리 (스시, 라멘 등)",
        "2. 현대적인 퓨전 요리",
        "3. 길거리 음식",
        "4. 고급 미슐랭 레스토랑"
    ]
    print("\n음식 선호도 (복수 선택 가능):")
    for pref in food_preferences:
        print(pref)
    food_choices = input("선호하는 음식 스타일 번호를 선택하세요 (쉼표로 구분): ").split(',')
    preferences['food'] = [int(choice.strip()) for choice in food_choices]
    
    place_preferences = [
        "1. 번화가와 쇼핑 지역",
        "2. 조용한 주택가와 공원",
        "3. 역사적인 명소와 신사",
        "4. 현대적인 건축물과 테크놀로지 중심지"
    ]
    print("\n장소 분위기 선호도 (복수 선택 가능):")
    for pref in place_preferences:
        print(pref)
    place_choices = input("선호하는 장소 분위기 번호를 선택하세요 (쉼표로 구분): ").split(',')
    preferences['place'] = [int(choice.strip()) for choice in place_choices]
    
    activity_preferences = [
        "1. 문화 체험 (예: 다도, 기모노 입기)",
        "2. 엔터테인먼트 (예: 디즈니랜드, 로봇 레스토랑)",
        "3. 자연 감상 (예: 공원, 정원)",
        "4. 쇼핑 (예: 면세점, 백화점)"
    ]
    print("\n활동 선호도:")
    for pref in activity_preferences:
        print(pref)
    activity_choice = input("가장 선호하는 활동 번호를 하나만 선택하세요: ")
    preferences['activity'] = int(activity_choice)
    
    # 우선순위 설정
    print("\n선호도 우선순위를 설정해주세요:")
    print("1. 음식, 2. 장소, 3. 활동")
    priority = input("우선순위 순서를 입력하세요 (예: 3,1,2): ").split(',')
    priority = [int(p) for p in priority]
    
    # 가중치 계산
    category_weights = {
        1: 0.5,  # 1순위
        2: 0.3,  # 2순위
        3: 0.2   # 3순위
    }
    
    # 수정된 가중치 계산 부분
    weights['food'] = category_weights[priority[0]] / len(preferences['food'])
    weights['place'] = category_weights[priority[1]] / len(preferences['place'])
    weights['activity'] = category_weights[priority[2]]
    
    return preferences, weights

# 고객 선호도 및 우선순위 반영
preferences, weights = get_customer_preferences()
travel_info["preferences"] = preferences
travel_info["weights"] = weights

# 태스크 정의
task1 = Task(
    description=f'''
    다음 여행 정보를 분석하고 고객의 선호도와 제약 사항을 파악하세요:
    - 목적지: {travel_info['destination']}
    - 기간: {travel_info['duration']}
    - 여행자: {travel_info['travelers']}
    - 도착 시간: {travel_info['arrival']}
    - 호텔: 1일차 {travel_info['hotels']['day1']}, 2-3일차 {travel_info['hotels']['day2_3']}
    - 출발 시간: {travel_info['departure']}
    - 선호사항: {travel_info['preferences']}
    - 가중치: {travel_info['weights']}
    고객의 선호도와 가중치를 고려하여 여행의 우선순위를 결정하세요.
    복수 선택된 음식과 장소 선호도에 대해 다양한 옵션을 제공하세요.
    최종결과를 한국어로 번역해주세요.
    ''',
    expected_output="고객의 여행 선호도와 제약 사항에 대한 상세한 분석, 우선순위가 반영된 계획 방향",
    agent=travel_planner
)

task2 = Task(
    description=f'''
    {travel_info['destination']}의 추천 장소와 활동 목록을 작성하세요. 
    다음 선호도와 가중치를 고려하여 {travel_info['duration']} 동안 
    {travel_info['travelers']}이 즐길 수 있는 장소를 추천해주세요:
    - 선호사항: {travel_info['preferences']}
    - 가중치: {travel_info['weights']}
    선택된 장소에 방문전 필수로 알아야 하는 TIP에 대해 정리해주세요
    최종결과를 한국어로 번역해주세요.
    ''',
    expected_output="도쿄 추천장소 목록, 방문전 필수Tip, 추천장소 별 활동 목록 (고객 선호도와 우선순위 반영)",
    agent=local_expert
)

task3 = Task(
    description=f'''
    주어진 정보와 추천 목록을 바탕으로 {travel_info['duration']}간의 세부 여행 일정을 최적화하세요.
    - 도착: {travel_info['arrival']}
    - 1일차 호텔: {travel_info['hotels']['day1']}
    - 2-3일차 호텔: {travel_info['hotels']['day2_3']}
    - 출발: {travel_info['departure']}
    다음 선호사항과 가중치를 고려하여 일정을 조정하세요:
    - 선호사항: {travel_info['preferences']}
    - 가중치: {travel_info['weights']}
    가중치가 높은 활동에 더 많은 시간을 할당하고, 각 동선별 이동경로와 이동수단을 상세히 알려주세요.
    복수 선택된 음식과 장소 선호도를 고려하여 다양한 경험을 제공하세요.
    음식점은 끼니별로 예비후보지 2~3곳도 추가로 알려주세요.
    최종결과를 한국어로 번역해주세요.
    ''',
    expected_output="3일간의 최적화된 세부 여행 일정 (우선순위와 가중치 반영), 장소별 음식점 예비후보지 2~3곳",
    agent=schedule_optimizer
)

# Crew 생성 및 실행
crew = Crew(
    agents=[travel_planner, local_expert, schedule_optimizer],
    tasks=[task1, task2, task3],
    verbose=1
)

result = crew.kickoff()

print("최종 여행 계획:")
print(result)


음식 선호도 (복수 선택 가능):
1. 일본 전통 요리 (스시, 라멘 등)
2. 현대적인 퓨전 요리
3. 길거리 음식
4. 고급 미슐랭 레스토랑


선호하는 음식 스타일 번호를 선택하세요 (쉼표로 구분):  1,2



장소 분위기 선호도 (복수 선택 가능):
1. 번화가와 쇼핑 지역
2. 조용한 주택가와 공원
3. 역사적인 명소와 신사
4. 현대적인 건축물과 테크놀로지 중심지


선호하는 장소 분위기 번호를 선택하세요 (쉼표로 구분):  1



활동 선호도:
1. 문화 체험 (예: 다도, 기모노 입기)
2. 엔터테인먼트 (예: 디즈니랜드, 로봇 레스토랑)
3. 자연 감상 (예: 공원, 정원)
4. 쇼핑 (예: 면세점, 백화점)


가장 선호하는 활동 번호를 하나만 선택하세요:  4



선호도 우선순위를 설정해주세요:
1. 음식, 2. 장소, 3. 활동


우선순위 순서를 입력하세요 (예: 3,1,2):  1,2,3


 [DEBUG]: == Working Agent: 여행 계획가


> Entering new CrewAgentExecutor chain...
Thought: 고객의 여행 선호도와 제약 사항을 분석하기 위해, 도쿄에서의 음식, 장소, 활동에 대한 정보를 수집해야 합니다. 이를 통해 최적의 여행 계획을 수립할 수 있습니다. 

Action: duckduckgo_search  
Action Input: {"query": "Tokyo food options 1 2"}  
 

Yakitori Toriyoshi in Nakameguro. 13. Oden 🍲🍢 (おでん) Oden is a type of nabemono, which is a soup that is served while still being simmered, similar to a hot pot. It often consists of different types and shapes of fishcakes, as well as radish, boiled eggs, and udon, simmered in a light soy and dashi-based broth. Rokurinsha is known for a type of ramen called Tsukemen style where soup and noodle are separated and the broth is much more rich and the noodles thicker. This is what they call "Special Dip Noodle" in English. Usually we're pretty open to trying new types of food but the Tsukemen style wasn't one for us. Nisshin Tasuke. Address: 〒104-0045 Tokyo, Chuo City, Tsukiji, 4 Chome−13−15 築地吉澤ビル 18 Closest Metro Station: Higashi

In [3]:
from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI
from langchain.tools import DuckDuckGoSearchRun
from langchain_anthropic import ChatAnthropic
from langchain.agents import Tool
from dotenv import load_dotenv
import os

load_dotenv()

gpt = ChatOpenAI(model="gpt-4o-mini",
             api_key=os.getenv("OPENAI_API_KEY"), 
             temperature=0,
             max_tokens=4000)

clud = ChatAnthropic(model="claude-3-5-sonnet-20240620",
                     anthropic_api_key=os.getenv("Anthropic_API_KEY"),
                     temperature=0,
                     max_tokens=4000)

# 도구 초기화
search_tool = DuckDuckGoSearchRun()

# 검색 도구 래퍼 함수
def search_wrapper(query):
    # 여기에서 쿼리 전처리나 로깅을 수행할 수 있습니다
    print(f"Performing search with query: {query}")
    return search_tool.run(query)

# 에이전트 정의
travel_planner = Agent(
    role='여행 계획가',
    goal='고객의 선호도와 예산에 맞는 최적의 여행 계획 수립',
    backstory='15년 경력의 전문 여행 계획가로, 다양한 고객의 요구사항을 만족시키는 맞춤형 여행 계획을 수립합니다.',
    verbose=True,
    allow_delegation=False,
    tools=[Tool(
        name="Search",
        func=search_wrapper,
        description="유용한 정보를 검색하는 도구"
    )],
    llm=gpt
)

local_expert = Agent(
    role='도쿄 현지 전문가',
    goal='도쿄의 숨겨진 명소와 현지 문화에 대한 깊이 있는 정보 제공',
    backstory='도쿄에서 10년 이상 거주한 현지 전문가로, 관광객들이 쉽게 접하기 어려운 특별한 경험을 제안합니다.',
    verbose=True,
    allow_delegation=False,
    tools=[Tool(
        name="Search",
        func=search_wrapper,
        description="유용한 정보를 검색하는 도구"
    )],
    llm=clud
)

schedule_optimizer = Agent(
    role='일정 최적화 전문가',
    goal='효율적이고 즐거운 3일 도쿄 여행 일정 최적화',
    backstory='데이터 분석과 여행 경험을 결합하여 고객의 시간과 에너지를 최대한 활용할 수 있는 최적의 일정을 설계합니다.',
    verbose=True,
    allow_delegation=False,
    tools=[Tool(
        name="Search",
        func=search_wrapper,
        description="유용한 정보를 검색하는 도구"
    )],
    llm=clud
)

# 나머지 코드는 동일하게 유지...